<a href="https://colab.research.google.com/github/GianmarcoLattaruolo/Vision_Project/blob/main/test_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
import os
cwd = os.getcwd()
os.chdir(cwd + r'\GeoEstimation')
sys.path.append(cwd + r'\GeoEstimation')
from pathlib import Path
from math import ceil

import pandas as pd
import torch
import pytorch_lightning as pl

from classification.train_base import MultiPartitioningClassifier # class defining our model
from classification.dataset import FiveCropImageDataset # class for preparing the images before giving them to the NN

## Load the model

In [3]:
#this cell is just to explore the number of attributes of the classes we have to work with
methods_MultiPar = [method_name for method_name in dir(MultiPartitioningClassifier)
                  if callable(getattr(MultiPartitioningClassifier, method_name))]
display(len(methods_MultiPar))

#MultiPartioningClassifier is child of pl.LightningModule
print(MultiPartitioningClassifier.__bases__)
methods_pl_Ligh = [method_name for method_name in dir(pl.LightningModule)
                  if callable(getattr(pl.LightningModule, method_name))]
display(len(methods_pl_Ligh))

#pl.LightningModule is child of torch.nn.modules.module.Module and several other PyTorch lightning classes
print(pl.LightningModule.__bases__)
methods_pytorch_nn = [method_name for method_name in dir(torch.nn.modules.module.Module)
                  if callable(getattr(torch.nn.modules.module.Module, method_name))]
display(len(methods_pytorch_nn))

#torch.nn.modules.module.Module is not a child class
print(torch.nn.modules.module.Module.__bases__)

#only 4 attributes/methods from MultiPartitioningClassifier are new w.r.t. pl.LightningModule
#but I guess some are overwritten
display(set(methods_MultiPar)-set(methods_pl_Ligh)) 
display(set(methods_pl_Ligh)-set(methods_MultiPar))

print("class attributes of pytorch_ligthning.LigthningModule:"+2*"\t",len(pl.LightningModule.__dict__.keys())) #class methods
print("class attributes of MultiparitioningClassifier:"+3*"\t",len(MultiPartitioningClassifier.__dict__.keys()))
M1 = pl.LightningModule()
print("instance attributes of pytorch_ligthning.LigthningModule:"+"\t",len(M1.__dict__.keys()))

148

(<class 'pytorch_lightning.core.lightning.LightningModule'>,)


144

(<class 'abc.ABC'>, <class 'pytorch_lightning.utilities.device_dtype_mixin.DeviceDtypeModuleMixin'>, <class 'pytorch_lightning.core.grads.GradInformation'>, <class 'pytorch_lightning.core.saving.ModelIO'>, <class 'pytorch_lightning.core.hooks.ModelHooks'>, <class 'pytorch_lightning.core.hooks.DataHooks'>, <class 'pytorch_lightning.core.hooks.CheckpointHooks'>, <class 'torch.nn.modules.module.Module'>)


68

(<class 'object'>,)


{'_MultiPartitioningClassifier__build_model',
 '_MultiPartitioningClassifier__init_partitionings',
 '_multi_crop_inference',
 'inference'}

set()

class attributes of pytorch_ligthning.LigthningModule:		 50
class attributes of MultiparitioningClassifier:			 18
instance attributes of pytorch_ligthning.LigthningModule:	 26


In [4]:
# where model's params and hyperparams are saved
checkpoint = "models/base_M/epoch=014-val_loss=18.4833.ckpt"
hparams = "models/base_M/hparams.yaml"
# load_from_checkpoint is a static method from pytorch lightning, inherited by MultiPartitioningClassifier
# it permits to load a model previously saved, in the form of a checkpoint file, and one with hyperparameters
# MultiPartitioningClassifier is the class defining our model
model = MultiPartitioningClassifier.load_from_checkpoint(
    checkpoint_path=checkpoint,
    hparams_file=hparams,
    map_location=None,
    stric = False #Whether to strictly enforce that the keys in checkpoint_path match
    # the keys returned by this module’s state dict.
)
#I put some the function's variables from the documentation, with some comments
wanted_precision = 32
trainer = pl.Trainer(callbacks=None, #Add a callback or list of callbacks.
                     gradient_clip_val=None, #The value at which to clip gradients. Passing gradient_clip_val=None disables gradient clipping
                     track_grad_norm= -1, #-1 = no track, otherwise tracks the p-norm. May be set to ‘inf’ infinity-norm. If using Automatic Mixed Precision (AMP), the gradients will be unscaled before logging them. 
                     check_val_every_n_epoch=1, # Perform a validation loop every after every N training epochs.
                     max_epochs=None, # Stop training once this number of epochs is reached. Disabled by default (None). If both max_epochs and max_steps are not specified, defaults to max_epochs = 1000. To enable infinite training, set max_epochs = -1.
                     max_steps = -1, #Stop training after this number of steps. 
                     log_every_n_steps=50, #How often to log within steps. Default: 50
                     accelerator=None, # different accelerator types (“cpu”, “gpu”, “tpu”, “ipu”, “hpu”, “mps, “auto”)
                     precision=wanted_precision, #Double precision (64), full precision (32), half precision (16) or bfloat16 precision (bf16).
                     resume_from_checkpoint=None, #Deprecated since version v1.5:use Trainer.fit(..., ckpt_path=...) instead.
                     auto_lr_find=False, #If set to True, will make trainer.tune() run a learning rate finder, trying to optimize initial learning for faster convergence.
                     auto_scale_batch_size=False) #If set to True, will initially run a batch size finder trying to find the largest batch size that fits into memory. 

GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores


In [21]:
csv10k = r'C:\Users\latta\GitHub\Vision_Project\GeoEstimation\resources\images\landscape10k\data_10k.csv'
new_data_urls = pd.read_csv(csv10k, sep = ';')
display(new_data_urls.head())
#ok è molto più difficile trovare le foto da questo file. L'url della foto ti spiaga come trovarlo qui
#https://www.flickr.com/services/api/misc.urls.html
#sometimes https://www.flickr.com/photos/{user-id}/{photo-id} works sometimes no
ss = 'https://www.flickr.com/photos/'
new_data_urls['url']=[ss + '/'.join((str(i[0]),str(i[1]))) for i in zip(new_data_urls['owner'], new_data_urls['photo_id'])]
for i in new_data_urls['url'].head(40):
    print(i)

# a questo punto imparando ad usare il loro file download_images.py dovremmo farcela

,photo_id,owner,gender,occupation,title,description,faves,lat,lon,u_city,u_country,taken,weather,season,daytime
0,17271526139,130418712@N05,1.0,NaN,Rio Trejo,Son numerosos los rios y arroyos que discurren...,701.0,"36,861544","-5,177747",NaN,NaN,2015-04-26 17:11:11,NaN,1.0,2.0
1,17776887679,55101137@N02,1.0,NaN,2015-05-13-022FD PH-XRD,<u><b>Aircraft Type - Registration - (c/n)</b>...,1.0,"51,463766","5,392935",Bodmin,United Kingdom,2015-05-13 00:00:22,9.0,1.0,3.0
2,17898331633,55101137@N02,1.0,NaN,2015-05-17-022FD OO-GWA,<u><b>Aircraft Type - Registration - (c/n)</b>...,2.0,"51,190492","4,453765",Bodmin,United Kingdom,2015-05-17 00:00:22,9.0,1.0,3.0
3,17940239919,55101137@N02,1.0,NaN,2015-05-14-020FD D-1553,<u><b>Aircraft Type - Registration - (c/n)</b>...,0.0,"51,326247","6,085953",Bodmin,United Kingdom,2015-05-14 00:00:20,9.0,1.0,3.0
4,17963122505,55101137@N02,1.0,NaN,2015-05-13-025FD EI-DLI,<u><b>Aircraft Type - Registration - (c/n)</b>...,2.0,"51,463766","5,392935",Bodmin,United Kingdom,2015-05-13 00:00:25,9.0,1.0,3.0


https://www.flickr.com/photos/130418712@N05/17271526139
https://www.flickr.com/photos/55101137@N02/17776887679
https://www.flickr.com/photos/55101137@N02/17898331633
https://www.flickr.com/photos/55101137@N02/17940239919
https://www.flickr.com/photos/55101137@N02/17963122505
https://www.flickr.com/photos/55101137@N02/18331250198
https://www.flickr.com/photos/69258414@N08/20052582226
https://www.flickr.com/photos/131084835@N03/20383414776
https://www.flickr.com/photos/133755874@N03/25740797653
https://www.flickr.com/photos/32675973@N00/25843872864
https://www.flickr.com/photos/130418712@N05/26503578166
https://www.flickr.com/photos/130418712@N05/27624094755
https://www.flickr.com/photos/69258414@N08/28853122201
https://www.flickr.com/photos/146106459@N03/29263898094
https://www.flickr.com/photos/130418712@N05/29499158291
https://www.flickr.com/photos/146106459@N03/29742232315
https://www.flickr.com/photos/146106459@N03/29777481322
https://www.flickr.com/photos/146106459@N03/29807819351


In [3]:
# I want to train on the second 3k-images test set
image_dir = r"resources\images\im2gps"
meta_csv = r"resources\images\im2gps_places365.csv"
#FiveCropImageDataset is the class for preparing the images before giving them to the NN
# in particular, it creates five different crops for every image
dataset = FiveCropImageDataset(meta_csv, image_dir)
# NOTA: in realtà il Five-Cropping avviene solo nel momento in cui si chiama dataset[idx]
# the authors created this classe from torch.utils.data.dataset.Dataset class
print(FiveCropImageDataset.__bases__)
batch_size = 64
#Data loader. Combines a dataset and a sampler, and provides single- or multi-process iterators over the dataset.
dataloader = torch.utils.data.DataLoader(
                    dataset = dataset, #dataset from which to load the data.
                    batch_size=ceil(batch_size / 5),  #you divide by 5 because for each image you generate 5 different crops
                    shuffle=False, # set to True to have the data reshuffled at every epoch (default: False).
                    num_workers=4, #number ot threads used for parallelism (cores of CPU?) 
                    #how many subprocesses to use for data loading. 0 means that the data will be loaded in the main process. (default: 0)
                    pin_memory=False, #If True, the data loader will copy tensors into CUDA pinned memory before returning them.
                    drop_last=False, #set to True to drop the last incomplete batch, if the dataset size is not divisible by the batch size.
                    timeout=0 # if positive, the timeout value for collecting a batch from workers. Should always be non-negative. (default: 0)               
                )

DIO BANANA
Read resources\images\im2gps_places365.csv
                                        img_id        author   latitude  \
0     104123223_7410c654ba_19_19355699@N00.jpg  19355699@N00 -16.663606   
1   1095548455_f636d22cbb_1277_8576809@N08.jpg   8576809@N08  31.893581   
2  1185597181_0158ab4213_1311_43616936@N00.jpg  43616936@N00  42.346571   
3  1199004207_0ce4e7a456_1285_16418049@N00.jpg  16418049@N00  37.090924   
4  1257001714_3453f5fc4b_1405_11490799@N08.jpg  11490799@N08  55.485759   

    longitude  s3_label  s16_label  s365_label  prob_indoor  prob_natural  \
0  145.563537         1          8         150     0.002959      0.777815   
1  -85.141124         2         15         231     0.003976      0.016128   
2  -71.097228         2         12         312     0.000005      0.000004   
3   25.370521         2         15         227     0.056002      0.007563   
4   28.791046         1          6         205     0.000083      0.991441   

   prob_urban  
0    0.219226  


In [5]:
#let's explore the dataset 
print(type(dataset))
print(len(dataset)) #forse sono 2997 e non 300 perchè hanno detto che non prendono più foto dello stesso autore...
print(type(dataset[0]))
print(len(dataset[0]))
print(type(dataset[0][0]))
print(type(dataset[0][1]))
print(dataset[0][1])
print(dataset[0][0].shape)
print(dataset.meta_info.head())
print(type(dataset.__getitem__(0)))
print(sum(sum(sum(sum(dataset[0][0]!=dataset.__getitem__(0)[0]))))) # __getimtem__ ti tira fuori la tupla di due elementi:
# il torch tensor dell'immagine e il dizionario dei vari dati (tipo gps) dell'immagine.
dataset.tfm

<class 'classification.dataset.FiveCropImageDataset'>
237
<class 'tuple'>
2
<class 'torch.Tensor'>
<class 'dict'>
{'img_id': '104123223_7410c654ba_19_19355699@N00.jpg', 'author': '19355699@N00', 'latitude': -16.663606, 'longitude': 145.56353700000003, 's3_label': 1, 's16_label': 8, 's365_label': 150, 'prob_indoor': 0.002959289950443811, 'prob_natural': 0.7778147804293098, 'prob_urban': 0.219225829974576, 'img_path': 'resources\\images\\im2gps\\104123223_7410c654ba_19_19355699@N00.jpg'}
torch.Size([5, 3, 224, 224])
                                        img_id        author   latitude  \
0     104123223_7410c654ba_19_19355699@N00.jpg  19355699@N00 -16.663606   
1   1095548455_f636d22cbb_1277_8576809@N08.jpg   8576809@N08  31.893581   
2  1185597181_0158ab4213_1311_43616936@N00.jpg  43616936@N00  42.346571   
3  1199004207_0ce4e7a456_1285_16418049@N00.jpg  16418049@N00  37.090924   
4  1257001714_3453f5fc4b_1405_11490799@N08.jpg  11490799@N08  55.485759   

    longitude  s3_label  s16_

Compose(
    ToTensor()
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
)

In [48]:
#we need to specify the validation data since we don't have the file:
#'resources/yfcc_25600_places365_mapping_h3.json'
c = 0.1 #ratio for validation set
val_data, train_data = torch.utils.data.random_split(dataset, [int(c*len(dataset)),len(dataset)-int(c*len(dataset))])
val_dataloader = torch.utils.data.DataLoader(
                    dataset = val_data, #dataset from which to load the data.
                    batch_size=ceil(batch_size / 5),  #you divide by 5 because for each image you generate 5 different crops
                    num_workers=4, #number ot threads used for parallelism (cores of CPU?) 
                    #how many subprocesses to use for data loading. 0 means that the data will be loaded in the main process. (default: 0)
                )
train_dataloader = torch.utils.data.DataLoader(
                    dataset = train_data, #dataset from which to load the data.
                    batch_size=ceil(batch_size / 5),  #you divide by 5 because for each image you generate 5 different crops
                    num_workers=4, #number ot threads used for parallelism (cores of CPU?) 
                    #how many subprocesses to use for data loading. 0 means that the data will be loaded in the main process. (default: 0)
                )
new_training = trainer.fit(model = model, #model to  fit
                           train_dataloader=train_dataloader, # Pytorch DataLoader with training samples. 
                           #If the model has a predefined train_dataloader method this will be skipped 
                           val_dataloaders=val_dataloader, #Either a single Pytorch Dataloader or a list of them, 
                           # specifying validation samples. If the model has a predefined val_dataloaders 
                           # method this will be skipped
                           datamodule=None)#A instance of LightningDataModule, optional
#this gives the error: ValueError: not enough values to unpack (expected 4, got 2) we both the 2 datasets we have.


  | Name       | Type       | Params
------------------------------------------
0 | model      | Sequential | 23 M  
1 | classifier | ModuleList | 47 M  
INFO:lightning:
  | Name       | Type       | Params
------------------------------------------
0 | model      | Sequential | 23 M  
1 | classifier | ModuleList | 47 M  


Validation sanity check: 0it [00:00, ?it/s]

ValueError: not enough values to unpack (expected 4, got 2)

# Some useful links

[load_from_checkpoints](https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.core.saving.ModelIO.html)

[pytorch.Trainer](https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html)

[transfer learning](https://pytorch-lightning.readthedocs.io/en/stable/advanced/finetuning.html)

[pytorch lightning 1.0.1 full documentation](https://pytorch-lightning.readthedocs.io/_/downloads/en/1.0.1/pdf/)
Unfortunately we need to watch this since several functions arguments have changed.